In [1]:
from nnsight import LanguageModel
from torch.nn import Module
from nnsight.util import WrapperModule
# import torch

/share/u/caden/.conda/envs/work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = LanguageModel("EleutherAI/pythia-70m", dispatch=True, device_map="auto")

/share/u/caden/.conda/envs/work/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
with model.trace("test") as tracer:
    print(tracer._backend)

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [3]:
from sae import AutoEncoder

sae = AutoEncoder.from_pretrained("./ae.pt", device="cuda:0")

In [4]:
class Edit(Module):

    def __init__(self):
        super().__init__()

        self.one = WrapperModule()
        self.two = WrapperModule()

    def forward(self, x):
        return self.two(self.one(x))
    
edit = Edit()

In [5]:
l0 = model.gpt_neox.layers[0]
l0._add_envoy(sae, "edit")

setattr(model._model.gpt_neox.layers[0], "edit", sae)

In [40]:
with model.trace("words", validate=False, scan=False):
    acts = l0.output[0]
    l0.output[0][:] = sae(acts)

    before = model.embed_out.output.save()

before.shape

torch.Size([1, 1, 50304])

In [41]:
with model.trace(validate=False) as tracer:

    with tracer.invoke("test"):

        acts = l0.output[0]

        l0.output[0][:] = l0.edit(acts)

        before = model.embed_out.output.save()

    with tracer.invoke("test"):

        acts = l0.output[0]

        l0.output[0][:] = l0.edit(acts)

        l0.edit.encoder.output *= 100

        after = model.embed_out.output.save()